<a href="https://colab.research.google.com/github/julianafrmendes/Business-Analytics/blob/master/Basket_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Business analytics

## Marketing Analysis

### Análise de vendas associadas e padrão de consumo com Pyspark


## Basket Analysis

A análise de cesta pode ser usada para identificar quais produtos são comprados juntos com frequência e, portanto, podem ser vendidos juntos para aumentar as vendas. Também pode ser usada para identificar produtos que são comprados com menos frequência, o que pode indicar problemas com esses produtos ou com sua apresentação nas prateleiras. É uma técnica de análise de dados frequentemente utilizada no varejo e no setor de bens de consumo.

In [25]:
!pip install -q -U watermark

In [26]:

# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Juliana Mendes, Data science academy" --iversions

Author: Juliana Mendes, Data science academy

numpy     : 1.21.6
seaborn   : 0.11.2
pyspark   : 3.3.1
pandas    : 1.3.5
matplotlib: 3.2.2



In [22]:
# Instala PySpark 
!pip install -q pyspark

In [23]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.fpm import FPGrowth
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, round
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [5]:
# Carregamos os dados processados e seguimos com o trabalho de análise
dados = pd.read_csv("DataCoSupplyChainDatasetProcessados.csv", encoding = 'latin1')

In [6]:
# Visualiza
dados.tail()

,Unnamed: 0,Type,Days for shipping (real),Days for shipment (scheduled),Sales per customer,Delivery Status,Category Id,Category Name,Customer City,Customer Country,...,Order Item Quantity,Sales,Order Item Total,Order Profit Per Order,Order Region,Order Status,Product Card Id,Product Name,Product Price,Customer Name
180514,180514,CASH,4,4,399.980011,Shipping on time,45,Fishing,Brooklyn,EE. UU.,...,1,399.980011,399.980011,40.000000,Eastern Asia,CLOSED,1004,Field & Stream Sportsman 16 Gun Fire Safe,399.980011,Maria Peterson
180515,180515,DEBIT,3,2,395.980011,Late delivery,45,Fishing,Bakersfield,EE. UU.,...,1,399.980011,395.980011,-613.770019,Eastern Asia,COMPLETE,1004,Field & Stream Sportsman 16 Gun Fire Safe,399.980011,Ronald Clark
180516,180516,TRANSFER,5,4,391.980011,Late delivery,45,Fishing,Bristol,EE. UU.,...,1,399.980011,391.980011,141.110001,Oceania,PENDING,1004,Field & Stream Sportsman 16 Gun Fire Safe,399.980011,John Smith
180517,180517,PAYMENT,3,4,387.980011,Advance shipping,45,Fishing,Caguas,Puerto Rico,...,1,399.980011,387.980011,186.229996,Oceania,PENDING_PAYMENT,1004,Field & Stream Sportsman 16 Gun Fire Safe,399.980011,Mary Smith
180518,180518,PAYMENT,4,4,383.980011,Shipping on time,45,Fishing,Caguas,Puerto Rico,...,1,399.980011,383.980011,168.949997,South Asia,PENDING_PAYMENT,1004,Field & Stream Sportsman 16 Gun Fire Safe,399.980011,Andrea Ortega


In [31]:
# Clientes únicos
dados['Customer Name'].unique()

array(['Cally Holloway', 'Irene Luna', 'Gillian Maldonado', ...,
       'Anika Davenport', 'Yuri Smith', 'Hyacinth Witt'], dtype=object)

 ### Associando clientes e produtos

In [32]:
#@title
# Vamos associar clientes e produtos
df = dados.copy()
df = df[['Customer Name', 'Product Name']]
df = df.drop_duplicates(['Customer Name', 'Product Name'])
df = df.groupby('Customer Name')['Product Name'].apply(list).reset_index(name = "Products")
df

,Customer Name,Products
0,Aaron Berger,[Under Armour Girls' Toddler Spine Surge Runni...
1,Aaron Boyle,[Diamondback Women's Serene Classic Comfort Bi...
2,Aaron Bush,"[Nike Men's CJ Elite 2 TD Football Cleat, Unde..."
3,Aaron Calhoun,"[Perfect Fitness Perfect Rip Deck, Pelican Sun..."
4,Aaron Carr,"[Pelican Sunstream 100 Kayak, Nike Men's CJ El..."
...,...,...
14028,Zorita Fisher,[Rock music]
14029,Zorita Frost,[Porcelain crafts]
14030,Zorita Mcbride,[Web Camera]
14031,Zorita Mcleod,[Summer dresses]


## Inicio da sessão spark e escolha do algorítmo de associação

In [9]:
#@title
# Cria sessão Spark
spark = SparkSession.builder.master("local").appName("DSA").config('spark.ui.port', '4050').getOrCreate()

In [ ]:
# Sessão
spark

In [10]:
# Cria o SQL Context
sqlContext = SQLContext(spark)

/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [11]:
# Converte o dataframe
df_spark_frame = sqlContext.createDataFrame(df)

In [12]:
type(df_spark_frame)

pyspark.sql.dataframe.DataFrame

In [13]:
df_spark_frame.show(5)

+-------------+--------------------+
|Customer Name|            Products|
+-------------+--------------------+
| Aaron Berger|[Under Armour Gir...|
|  Aaron Boyle|[Diamondback Wome...|
|   Aaron Bush|[Nike Men's CJ El...|
|Aaron Calhoun|[Perfect Fitness ...|
|   Aaron Carr|[Pelican Sunstrea...|
+-------------+--------------------+
only showing top 5 rows



In [15]:
# Crie o objeto fpGrowth instanciando o construtor FPGrowth com os parâmetros necessários
fpGrowth = FPGrowth(itemsCol = "Products", minSupport = 0.015, minConfidence = 0.70)

In [16]:
# Ajusta o dataframe no objeto fpGrowth para preparar o modelo
modelo = fpGrowth.fit(df_spark_frame)

In [17]:
modelo

FPGrowthModel: uid=FPGrowth_fb3be8962c77, numTrainingRecords=14033

## Os itens que ocorrem com mais frequência

In [18]:
# Exibir os itens que ocorrem com mais frequência
modelo.freqItemsets.sort('freq', ascending = False).show(truncate = False)

+------------------------------------------------------------------------------------+----+
|items                                                                               |freq|
+------------------------------------------------------------------------------------+----+
|[Perfect Fitness Perfect Rip Deck]                                                  |4851|
|[Nike Men's CJ Elite 2 TD Football Cleat]                                           |4711|
|[Nike Men's Dri-FIT Victory Golf Polo]                                              |4603|
|[O'Brien Men's Neoprene Life Vest]                                                  |4505|
|[Field & Stream Sportsman 16 Gun Fire Safe]                                         |4304|
|[Nike Men's CJ Elite 2 TD Football Cleat, Perfect Fitness Perfect Rip Deck]         |4135|
|[Pelican Sunstream 100 Kayak]                                                       |4108|
|[Nike Men's Dri-FIT Victory Golf Polo, Perfect Fitness Perfect Rip Deck]       

## Tabela de associação

* Support -- Esta medida dá uma ideia da frequência de um conjunto de itens em todas as transações.

* Confidence -- Essa medida define a probabilidade de ocorrência de consequentes no carrinho, uma vez que o carrinho já possui os antecedentes. 

* Lift --  Representa aumento que {X} proporciona à nossa confiança por ter {Y} no carrinho.

In [19]:
#@title
# Mostra as regras de associação geradas
modelo.associationRules\
.withColumn("confidence", round(col("confidence"), 3))\
.withColumn("lift", round(col("lift"), 3))\
.withColumn("support", round(col("support"),10))\
.sort('confidence', ascending = False)\
.show(truncate = True)

+--------------------+--------------------+----------+-----+------------+
|          antecedent|          consequent|confidence| lift|     support|
+--------------------+--------------------+----------+-----+------------+
|[Team Golf St. Lo...|[Nike Men's CJ El...|     0.982|2.924|0.0151785078|
|[Team Golf St. Lo...|[Nike Men's CJ El...|     0.978|2.913|0.0158198532|
|[Titleist Pro V1 ...|[Perfect Fitness ...|     0.977|2.827|0.0153922896|
|[Team Golf St. Lo...|[Nike Men's CJ El...|     0.977|2.911|0.0152497684|
|[Titleist Pro V1 ...|[Nike Men's CJ El...|     0.977|2.911|0.0153922896|
|[ENO Atlas Hammoc...|[Perfect Fitness ...|     0.977|2.826|0.0150359866|
|[ENO Atlas Hammoc...|[Perfect Fitness ...|     0.974|2.817|0.0158198532|
|[Titleist Pro V1x...|[Perfect Fitness ...|     0.973|2.814| 0.015321029|
|[Team Golf St. Lo...|[Nike Men's CJ El...|     0.973|2.898| 0.015321029|
|[Titleist Pro V1 ...|[Nike Men's Dri-F...|     0.973|2.966|0.0153922896|
|[Team Golf St. Lo...|[Nike Men's CJ E

A tabela representa a relação de  consumo e o padrão de compra relacionada com um determinado produto, para o estudo foi eleito um grau de confiança mínima  de 70%, para que aumente a evidência da ocorrência de combinações entre os produtos antecessores e seus consequentes. 

# Fim